Este codigo esta hecho en base a una pregunta en StackOverflow (https://stackoverflow.com/questions/76623208/run-threadpool-on-multiple-spark-nodes) da un ejemplo sobre como se puede distribuir una tarea _multiproceso_ usando Spark y a la vez _multihilo_ dentro de cada nodo usando ThreadPool

In [1]:
# inicializamos una sesion de Spark

from pyspark.sql import SparkSession
# local[N cantidad de cores]
spark = SparkSession.builder \
    .master("local[4]") \
    .appName("Spark + ThreadPool") \
    .getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import time

def multithread(task, l):
   with ThreadPoolExecutor() as executor:
      results = list(executor.map(task, l))
   return results

def square(x):
   time.sleep(1)
   return x**2

def partition(l, n):
   # this function just partitions an input list into 'n' chunks
   for i in range(0, len(l), n):
      yield l[i:i +n]

num = list(range(100))
workers = 10
chunks = list(partition(num, workers))
rdd = spark.parallelize(chunks, numSlices=workers)
results = rdd.flatMap(lambda x: multithread(square, x)).collect()